Here is a way to extract tweets based on certain topics from Twitter API using the Tweepy library.
For using the Twitter API you need to have a developer access Twitter account. Once, you’re done with the set up create an app, in it, you will get Keys and tokens, which will help us retrieve data from Twitter. They act as login credentials.

Now, we will get on to code to get the tweets. First, you need to import all the packages required and initialize the token and key variables. OAuth essentially allows the user, via an authentication provider that they have previously successfully authenticated with, to give another website/service a limited access authentication token for authorization to additional resources.

In [1]:
#import files and give access to tokens and keys
import tweepy,json
access_token="1428093851726663696-55Xp7zjUQknkuPRDzCObGCcIQuxmOh"
access_token_secret="1jxsg4RdkszPMURIRi78aNm2hoHwMzz4VrBU69gmOV0gm"
consumer_key="9LanGta2OOzJBITyWRacIyxUR"
consumer_secret="mAfvIGHwHblw0bMJfszyn2voADCUWLEAT1ezbgPPiZb1geHi8X"
auth= tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret)

After getting access to Twitter data we’ll now create a file to save all the tweets in it.

In [2]:
tweet_list=[]
class MyStreamListener(tweepy.StreamListener):
    def __init__(self,api=None):
        super(MyStreamListener,self).__init__()
        self.num_tweets=0
        self.file=open("tweet.txt","w")
    def on_status(self,status):
        tweet=status._json
        self.file.write(json.dumps(tweet)+ '\n')
        tweet_list.append(status)
        self.num_tweets+=1
        if self.num_tweets<100:
            return True
        else:
            return False
        self.file.close()

We’ll now create a filter that will extract tweets based on certain words that are mentioned. Basically, it will extract tweets that contain the words which are valid for our project. For example, if you want data regarding covid19 you will use specific words like corona, coronavirus, covid19, etc to filter out the tweets as shown below.

In [3]:
#create streaming object and authenticate
l = MyStreamListener()
stream =tweepy.Stream(auth,l)
#this line filters twiiter streams to capture data by keywords
stream.filter(track=['AMD'])

You can read the tweets stored in the file by following:-

In [4]:
tweets_data_path='tweet.txt'
tweets_data=[]
tweets_file=open(tweets_data_path,"r")
#read in tweets and store on list
for line in tweets_file:
    tweet=json.loads(line)
    if tweet["lang"] == 'en':
        tweets_data.append(tweet)
tweets_file.close()
print(json.dumps(tweets_data[0], indent=4, sort_keys=True))

{
    "contributors": null,
    "coordinates": null,
    "created_at": "Fri Aug 20 20:07:07 +0000 2021",
    "entities": {
        "hashtags": [],
        "symbols": [
            {
                "indices": [
                    68,
                    73
                ],
                "text": "AAPL"
            },
            {
                "indices": [
                    91,
                    96
                ],
                "text": "TSLA"
            }
        ],
        "urls": [
            {
                "display_url": "twitter.com/i/web/status/1\u2026",
                "expanded_url": "https://twitter.com/i/web/status/1428810849632804875",
                "indices": [
                    114,
                    137
                ],
                "url": "https://t.co/MGm5p7CqOF"
            }
        ],
        "user_mentions": []
    },
    "extended_tweet": {
        "display_text_range": [
            0,
            275
        ],
        "entities": {

In [13]:
import re
import os
import json
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob
from pymongo import MongoClient

In [14]:
class TwitterClient(object):
    '''
    Generic Twitter Class for sentiment analysis
    '''
    def __init__(self):
        '''
        Class constructor method.
        '''
        # keys and tokens from the Twitter Dev Console
        consumer_key = "9LanGta2OOzJBITyWRacIyxUR"
        consumer_secret = "mAfvIGHwHblw0bMJfszyn2voADCUWLEAT1ezbgPPiZb1geHi8X"
        access_token = "1428093851726663696-55Xp7zjUQknkuPRDzCObGCcIQuxmOh"
        access_token_secret = "1jxsg4RdkszPMURIRi78aNm2hoHwMzz4VrBU69gmOV0gm"
        
        
        # attempt authentication
        try:
            # create OAuthHandler object
            self.auth = OAuthHandler(consumer_key, consumer_secret)
            # set access token and secret
            self.auth.set_access_token(access_token, access_token_secret)
            # create tweepy API object to fetch tweets
            self.api = tweepy.API(self.auth)
        except:
            print("Error: Authentication Failed")
    
    def clean_tweet(self, tweet):
        '''
        Utility function to clean tweet text by removing links, special characters
        using simple regex statements.
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
  
    def get_tweet_sentiment(self, tweet):
        '''
        Utility function to classify sentiment of passed tweet
        using textblob's sentiment method
        '''
        # create TextBlob object of passed tweet text
        analysis = TextBlob(self.clean_tweet(tweet))
        # set sentiment
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'
        
    def get_tweets(self, query, count = 10):
        '''
        Main function to fetch tweets and parse them.
        '''
        # empty list to store parsed tweets
        tweets = []
  
        try:
            # call twitter api to fetch tweets
            fetched_tweets = self.api.search(q = query, count = count)
  
            # parsing tweets one by one
            for tweet in fetched_tweets:
                # empty dictionary to store required params of a tweet
                parsed_tweet = {}
  
                # saving text of tweet
                parsed_tweet['text'] = tweet.text
                # saving sentiment of tweet
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)
  
                # appending parsed tweet to tweets list
                if tweet.retweet_count > 0:
                    # if tweet has retweets, ensure that it is appended only once
                    if parsed_tweet not in tweets:
                        tweets.append(parsed_tweet)
                else:
                    tweets.append(parsed_tweet)
  
            # return parsed tweets
            return tweets
  
        except tweepy.TweepError as e:
            # print error (if any)
            print("Error : " + str(e))

In [15]:
def main():
    # creating object of TwitterClient Class
    api = TwitterClient()
    # calling function to get tweets
    tweets = api.get_tweets(query = 'AAPL', count = 200)
    
    tweetsJSON = json.dumps(tweets)
    print('type',type(tweetsJSON))
    print(tweetsJSON)
    
    path = '/Users/tenzintashi/Downloads/Numeraxial/Numeraxial-Internship/backend/data/'
    filename = 'twitter_sentiment.json'
    with open(os.path.join(path,filename), 'w+', encoding='utf-8') as f:
        f.write(tweetsJSON)
  
#     # picking positive tweets from tweets
#     ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive']
#     # percentage of positive tweets
#     print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets)))
#     # picking negative tweets from tweets
#     ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']
#     # percentage of negative tweets
#     print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets)))
#     # percentage of neutral tweets
#     print("Neutral tweets percentage: {} % \
#         ".format(100*(len(tweets) -(len( ntweets )+len( ptweets)))/len(tweets)))
  
#     # printing first 5 positive tweets
#     print("\n\nPositive tweets:")
#     for tweet in ptweets[:10]:
#         print(tweet['text'])
  
#     # printing first 5 negative tweets
#     print("\n\nNegative tweets:")
#     for tweet in ntweets[:10]:
#         print(tweet['text'])
  
if __name__ == "__main__":
    # calling main function
    main()

type <class 'str'>
[{"text": "$AAPL starting to get a move.  \ud83d\udc40 ATH very possible this week!", "sentiment": "neutral"}, {"text": "RT @shan_the_man_34: Updated Portfolio by Weight:\n$SE $AMZN $SOFI $PLTR $AAPL $UPST $TSLA $CRWD $DMTK $LCID $ABNB", "sentiment": "neutral"}, {"text": "Base-upon-base is a powerful pattern. That's what $AAPL is building currently. A positive court decision in the Epi\u2026 https://t.co/UfnJjq8nNL", "sentiment": "negative"}, {"text": "Approval ~2 Yrs Before Effectiveness Study Completion\n$aapl $ba $bhc $crm $dis $f $ge $ko $brk $amzn $kr $tst $wmt\u2026 https://t.co/aKuuhrf4Vs", "sentiment": "neutral"}, {"text": "$TSLA $AAPL $FUBO $NVDA \n\n\ud83c\udf08\ud83d\udc3b https://t.co/ZVUx6JZfA0", "sentiment": "neutral"}, {"text": "Updated Portfolio by Weight:\n$SE $AMZN $SOFI $PLTR $AAPL $UPST $TSLA $CRWD $DMTK $LCID $ABNB", "sentiment": "neutral"}, {"text": "AAPL: 149.32 at Aug 23, 2021 2:23 PM", "sentiment": "neutral"}, {"text": "$AAPL some good flow t